In [1]:
push!(LOAD_PATH,".")
using Pkg
Pkg.add("DataStructures")

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [2]:
using Typeside, Chakra, PitchADT, MidiPitch, ChakraImp, Viewpoints, ContextModels

In [3]:
o1 = set(delim(),:pitch,NoteNumber(60))
o2 = set(delim(),:pitch,NoteNumber(62))
o3 = set(delim(),:pitch,NoteNumber(68))
o4 = set(delim(),:pitch,NoteNumber(60))
o5 = set(delim(),:pitch,NoteNumber(62))
o6 = set(delim(),:pitch,NoteNumber(70))

ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pitch => NoteNumber(70)))

In [4]:
s = [o1,o2,o3,o4,o5,o6]

6-element Vector{ChakraImp.Obj}:
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pitch => NoteNumber(60)))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pitch => NoteNumber(62)))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pitch => NoteNumber(68)))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pitch => NoteNumber(60)))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pitch => NoteNumber(62)))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pitch => NoteNumber(70)))

In [5]:
v = AtomicViewpoint(Pitch,:pitch)
vp_type(v)
vp_apply(v,s)

NoteNumber(60)

In [6]:
v2 = LinkedViewpoint(v,DelayedViewpoint(v,1))
#vp_type(v2)
vp_apply(v2,s)

(NoteNumber(60), NoteNumber(62))

In [7]:
v3 = DerivedViewpoint(PitchInterval,v2,PitchADT.diff)
vp_type(v3)
vp_apply(v3,s)

NoteInterval(2)

In [8]:
v4 = DerivedViewpoint(Pitch,v3,x->PitchADT.shift(x,NoteNumber(40)))
vp_type(v4)
vp_apply(v4,s)

NoteNumber(42)

In [9]:
m = ContextModels.construct_model(s,v,3)

ContextModels.ContextModel{Pitch}(Dict{Vector{Pitch}, Int64}([NoteNumber(60), NoteNumber(62), NoteNumber(70)] => 1, [NoteNumber(62)] => 2, [NoteNumber(60), NoteNumber(62)] => 2, [NoteNumber(60), NoteNumber(62), NoteNumber(68)] => 1, [] => 6, [NoteNumber(70)] => 1, [NoteNumber(60)] => 2, [NoteNumber(62), NoteNumber(70)] => 1, [NoteNumber(68), NoteNumber(60), NoteNumber(62)] => 1, [NoteNumber(60), NoteNumber(62), NoteNumber(68), NoteNumber(60)] => 1…), Set(Pitch[NoteNumber(68), NoteNumber(60), NoteNumber(70), NoteNumber(62)]))

In [10]:
ContextModels.probability_of(NoteNumber(68),[NoteNumber(60),NoteNumber(62)],m)

0.5

In [11]:
ContextModels.probability_of(NoteNumber(80),[NoteNumber(60)],m)

0.0

In [12]:
m.elems

Set{Pitch} with 4 elements:
  NoteNumber(68)
  NoteNumber(60)
  NoteNumber(70)
  NoteNumber(62)

In [13]:
m.db

Dict{Vector{Pitch}, Int64} with 16 entries:
  [NoteNumber(60), NoteNumber(62), NoteNumber(70)]                 => 1
  [NoteNumber(62)]                                                 => 2
  [NoteNumber(60), NoteNumber(62)]                                 => 2
  [NoteNumber(60), NoteNumber(62), NoteNumber(68)]                 => 1
  []                                                               => 6
  [NoteNumber(70)]                                                 => 1
  [NoteNumber(60)]                                                 => 2
  [NoteNumber(62), NoteNumber(70)]                                 => 1
  [NoteNumber(68), NoteNumber(60), NoteNumber(62)]                 => 1
  [NoteNumber(60), NoteNumber(62), NoteNumber(68), NoteNumber(60)] => 1
  [NoteNumber(68), NoteNumber(60), NoteNumber(62), NoteNumber(70)] => 1
  [NoteNumber(62), NoteNumber(68)]                                 => 1
  [NoteNumber(62), NoteNumber(68), NoteNumber(60)]                 => 1
  [NoteNumber(68), N